In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import collections
from collections import Counter
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from datetime import date
import time

## Lemmatisieren und Stoppwörter entfernen
1. Lemmatisieren des Textes
2. Umlaute umwandeln
3. Sonderzeichen und Bindestriche entfernen
4. Standard Stoppwörter entfernen
5. Text kleinschreiben

**Dieser Abschnitt des Codes ist stark am Code von Latifi et al. (2024) orientiert. Dieser ist hier zu finden:** https://github.com/albi-lt/Fiscal-Policy-in-the-Bundestag/tree/main

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('german'))

Benötigte Funktionen.

In [ ]:
def convert_umlauts(dataframe,textcolumn):
    dataframe[textcolumn].replace('Ä','AE',regex=True, inplace = True)
    dataframe[textcolumn].replace('ä','ae',regex=True, inplace = True)
    dataframe[textcolumn].replace('Ü','UE',regex=True, inplace = True)
    dataframe[textcolumn].replace('ü','ue',regex=True, inplace = True)
    dataframe[textcolumn].replace('Ö','OE',regex=True, inplace = True)
    dataframe[textcolumn].replace('ö','oe',regex=True, inplace = True)
    dataframe[textcolumn].replace('ß','ss',regex=True, inplace = True)

    return dataframe

def convert_umlauts_strings(words):
    mapping = {ord(u"Ü"): u"Ue", ord(u"ü"): u"ue", ord(u"ß"): u"ss", ord(u"ä"): u"ae", ord(u"Ä"): u"Ae",
               ord(u"ö"): u"oe", ord(u"Ö"): u"Oe"}
    # Übersetze jeden Eintrag in der Liste
    converted_words = [word.translate(mapping) for word in words]

    return converted_words

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True,max_len = 30))
        # Token mit nur einem oder mehr als 30 Zeichen werden exkludiert

def remove_words(texts,words):
    docs = []
    for doc in tqdm(texts):
        doc_cleaned = list(filter(lambda word: word.lower() not in words, doc.split()))
        docs.append(' '.join(doc_cleaned))
    return docs

def remove_stopwords(texttoken,stopwordslist):
    filtered_text = [word for word in texttoken if word.lower() not in stopwordslist]
    return filtered_text

def lemmatize_texts(texts, spacy_model):
    nlp_model = spacy.load(spacy_model)
    pp = []
    print('Start lemmatization...')
    t0 = time.time()
    for i in tqdm(range(len(texts))):
        text = " ".join([token.lemma_ for token in nlp_model(texts[i])])#
        pp.append(text)
    t1 = time.time()
    print('Finished lemmatization. Process took', t1 - t0, 'seconds')
    return pp

def remove_specific_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in lemmatized_stopwords]
    return ' '.join(filtered_words)

In [ ]:
data_all_lp = pd.read_csv('all_bundestag_speeches_replication_data.csv', index_col = 'Unnamed: 0')

'''
Die Reden sind ebenfalls auf dem GitHub von Latifi zu finden.
'''

data_all_lp.Role.unique()

# Limitieren der Daten auf den Zeitraum ab 2000
data_all_lp['Datum'] = pd.to_datetime(data_all_lp['date'], format='%d.%m.%Y')
data_all_lp = data_all_lp.loc[data_all_lp['Datum'].dt.year >= 2000]

# Filtern der Reden. Behalten werden: 'MdB','Bundeskanzler','Bundesminister','Staatssekretär', 'Staatsminister'
data_filtered = data_all_lp[data_all_lp.Role.isin(['MdB','Bundesminister','Bundeskanzler','Staatssekretär','Staatsminister'])]

print(len(data_filtered))
data_filtered

In [ ]:
pd.DataFrame(data_filtered.text_length).describe(percentiles = [0.05,0.1,0.15,0.25,0.5,0.75,0.8,0.9,0.95,0.99,0.995])

# Zu lange und zu kurze Reden werden entfernt
data_filtered = data_filtered[(data_filtered.text_length >= 100) & (data_filtered.text_length <= np.quantile(data_filtered.text_length,0.995))].reset_index(drop=True)
print(len(data_filtered))
data_filtered

### Lemmatisieren

In [ ]:
# Lemmatisieren der Texte
lemmatized = lemmatize_texts([data_filtered.text[i] for i in range(data_filtered.shape[0])], 'de_core_news_lg')

# Erstellen einer neuen Spalte mit den lemmatisierten Texten
data_filtered['lemmatized_text'] = lemmatized

# Ersetzen von Zeilenumbrüchen innerhalb von Wörtern
data_lemmatized = data_filtered.replace('-\n', '', regex=True)

# Speichern der Daten als CSV-Datei mit der neuen Spalte
data_lemmatized.to_csv('data_lemmatized.csv', index=False)

In [ ]:
# Ersetzen von Zeilenumbrüchen in der neuen Spalte 'lemmatized_text'
data_lemmatized['lemmatized_text'] = data_filtered['lemmatized_text'].replace('-\n', '', regex=True)

# Umwandeln von Umlauten in alternative Schreibweise
data_lemmatized['lemmatized_text'] = convert_umlauts_strings(data_lemmatized['lemmatized_text'])

In [ ]:
# Ersetze Zeilenumbrüche in der neuen Spalte 'lemmatized_text'
data_lemmatized['lemmatized_text'] = data_lemmatized['lemmatized_text'].replace('-\n', '', regex=True)

### Entfernen von nltk-Stoppwörtern

In [ ]:
# Entfernen von Standard Stoppwörtern
data_lemmatized['text_preprocessed'] = data_lemmatized['text_preprocessed'].apply(
    lambda words: [word for word in words if word.lower() not in stop_words]
)

In [ ]:
# Funktion zum Entfernen der Leerzeichen zwischen Buchstaben
def remove_spaces_in_words(word_list):
    # Entfernen von Leerzeichen aus jedem "Wort"
    return ["".join(word.split()) for word in word_list]

# Anwenden der Funktion auf die Spalte 'text_preprocessed'
data_lemmatized['text_preprocessed'] = data_lemmatized['text_preprocessed'].apply(remove_spaces_in_words)

In [ ]:
# Umbenennen von Spaltennamen
data_lemmatized.rename(columns={'period': 'periode'}, inplace=True)
data_lemmatized.rename(columns={'governing_Party': 'regierung'}, inplace=True)
data_lemmatized.rename(columns={'Party': 'partei'}, inplace=True)
data_lemmatized.rename(columns={'date': 'Datum'}, inplace=True)
data_lemmatized.rename(columns={'text_length': 'länge_vorher'}, inplace=True)

In [ ]:
# Vorverarbeitete Reden in einer neuen .csv speichern
data_lemmatized[['doc_id', 'doc_lp_id', 'speech_identification_ent', 'Datum', 'period', 'governing_Party', 'text_preprocessed', 'Party']].to_csv(
    'data_preprocessed.csv',
    index=False
)

In [ ]:
text_length_preprocessed = [len(i.split()) for i in data_lemmatized.text_preprocessed] #counts document lengths after removing stopwords
# text_length_lemmatized = [len(i.split()) for i in data_lemmatized.text_preprocessed_lemmatized]

data_lemmatized['länge_bereinigt'] = text_length_preprocessed
# data_lemmatized['text_length_lemmatized'] = text_length_lemmatized

data_lemmatized['datum_jahr'] = data_lemmatized.date.dt.year
data_lemmatized['datum_quartal'] = data_lemmatized.date.dt.quarter

data_lemmatized[['doc_id', 'doc_lp_id', 'speech_identification_ent', 'Datum', 'datum_jahr', 'datum_quartal', 'period', 'governing_Party', 'text_preprocessed', 'länge_vorher', 'länge_bereinigt', 'Party']].to_csv(
    'speeches_preprocessed.csv',
    index=False
)

In [ ]:
speeches_preprocessed.rename(columns={'period': 'periode'}, inplace=True)
speeches_preprocessed.rename(columns={'date_year': 'datum_jahr'}, inplace=True)
speeches_preprocessed.rename(columns={'date_quarter': 'datum_quartal'}, inplace=True)
speeches_preprocessed.rename(columns={'governing_Party': 'regierung'}, inplace=True)
speeches_preprocessed.rename(columns={'Party': 'partei'}, inplace=True)
speeches_preprocessed.rename(columns={'date': 'Datum'}, inplace=True)
speeches_preprocessed.rename(columns={'text_length': 'länge_vorher'}, inplace=True)
speeches_preprocessed.rename(columns={'text_length_preprocessed': 'länge_bereinigt'}, inplace=True)

In [ ]:
länge_original = speeches_preprocessed['länge_vorher'].sum()
länge_preprocessed = speeches_preprocessed['länge_bereinigt'].sum()

entfernte_wörter = länge_original - länge_preprocessed

print(f"Vor dem Entfernen allgemeiner Stoppwörter umfasste der Datensatz {länge_original} Wörter.")
print(f"Durch das Entfernen entfielen {entfernte_wörter} Wörter.")
print(f"Somit umfasst der Korpus jetzt noch {länge_preprocessed} Wörter.")

### Entfernen spezifischer Stoppwörter

Die Stoppwörter basieren auf der Liste von Latifi et al., (2024) und wurden manuell angepasst.

In [ ]:
# Schritt 1: Einlesen der CSV mit spezifischen Stoppwörtern
stopwords_df = pd.read_csv('self_filtered_stopwords_german_bundestag.csv', header=None)

# Schritt 2: Extrahieren der lemmatisierten Stoppwörter aus der Tabelle
lemmatized_stopwords = stopwords_df[2].drop(0).tolist()

# Anwenden der Funktion auf die Spalte 'text_preprocessed'
speeches_preprocessed['text_preprocessed_specific'] = speeches_preprocessed['text_preprocessed'].apply(remove_specific_stopwords)

# Wörter nach der Entfernung spezifischer Stoppwörter zählen
länge_specific = speeches_preprocessed['text_preprocessed_specific'].apply(lambda x: len(str(x).split())).sum()

# Schritt 3: Ergebnisse ausgeben
entfernte_wörter_specific = länge_preprocessed - länge_specific
print(f"Vor dem Entfernen spezifischer Stoppwörter umfasste der Datensatz {länge_preprocessed} Wörter.")
print(f"Durch das Entfernen entfielen {entfernte_wörter_specific} Wörter.")
print(f"Somit umfasst der Korpus jetzt noch {länge_specific} Wörter.")

In [ ]:
text_length_specific = [len(i.split()) for i in speeches_preprocessed.text_preprocessed_specific] # Textlänge nach dem Bereinigen der Artikel

speeches_preprocessed['länge_bereinigt_spezifisch'] = text_length_specific

# Speichern in einer neuen .csv Datei
speeches_preprocessed.to_csv('bereinigte_reden_spezifisch.csv', index=False)

### Graphische Darstellung

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Definieren von Bins für Gruppen von 20 Tokens
bin_edges = range(0, speeches_preprocessed["länge_vorher"].max() + 20, 20)

# Frequenzberechnung für jede Textlängenspalte
freq_original = pd.cut(speeches_preprocessed["länge_vorher"], bins=bin_edges).value_counts().sort_index()
freq_nltk = pd.cut(speeches_preprocessed["länge_bereinigt"], bins=bin_edges).value_counts().sort_index()
freq_specific = pd.cut(speeches_preprocessed["länge_bereinigt_spezifisch"], bins=bin_edges).value_counts().sort_index()

# Mittelpunkte der Bins
bin_midpoints = [interval.mid for interval in freq_original.index.categories]

# Plot
plt.figure(figsize=(12, 6))
plt.bar(bin_midpoints, freq_original, width=18, label="Originale Länge der Reden", alpha=0.7, color='blue')
plt.bar(bin_midpoints, freq_nltk, width=18, label="Länge nltk-Stoppwörter entfernt", alpha=0.7, color='orange')
plt.bar(bin_midpoints, freq_specific, width=18, label="Länge spezifische Stoppwörter entfernt", alpha=0.7, color='green')

# Achsenbeschriftungen und Titel
plt.xlabel("Anzahl der Wörter in der Rede", fontsize=12)
plt.ylabel("Anzahl der Reden", fontsize=12)
plt.title("", fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('EDA_reden_1.png', format='png', dpi=600)

plt.show()